In [2]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=481f101d2925931c46e8679b2006bc55f2054d3f3ceab65dc101fbbb285e70d4
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [3]:
!pip install transformers

     |████████████████████████████████| 542kB 7.2MB/s 
     |████████████████████████████████| 3.7MB 57kB/s 
     |████████████████████████████████| 1.0MB 41.8MB/s 
     |████████████████████████████████| 870kB 43.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=bf1d3c416ed9315670d575ef314024196fd4b4ed947ee63aff683e5d7811ccba
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [4]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [5]:
import torch
if torch.cuda.is_available():     # If there's a GPU available...        
    device = torch.device("cuda")   # Tell PyTorch to use the GPU.
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [0]:
import tensorflow as tf

In [7]:
import wget
import os

print('Downloading dataset...')
url = 'http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv'
 
if not os.path.exists('./qqp2.tsv'):
    wget.download(url, './qqp2.tsv')

In [8]:
os.listdir()

['.config', 'qqp2.tsv', 'sample_data']

In [9]:
import pandas as pd
file = pd.read_csv('qqp2.tsv', delimiter='\t')
file.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [0]:
q1 = file['question1'].values[1:10000]
q2 = file['question2'].values[1:10000]
labels = file['is_duplicate'].values[1:10000]

In [11]:
from transformers import BertTokenizer
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [59]:
t1 = tokenizer.tokenize(q1[0])
t2 = tokenizer.tokenize(q2[0])
id1 = tokenizer.convert_tokens_to_ids(t1)
id2 = tokenizer.convert_tokens_to_ids(t2)
ip_eg = tokenizer.build_inputs_with_special_tokens(id1, id2)
seg_mask_eg = tokenizer.create_token_type_ids_from_sequences(id1, id2)

print(' Original: ', q1[10], q2[10])
print('Tokenized: ', t1, t2)
print('Token IDs: ', id1, id2)
print("Combined: ", ip_eg)
print('Segmentation Mask: ', seg_mask_eg)

 Original:  How do I read and find my YouTube comments? How can I see all my Youtube comments?
Tokenized:  ['what', 'is', 'the', 'story', 'of', 'ko', '##hin', '##oor', '(', 'ko', '##h', '-', 'i', '-', 'no', '##or', ')', 'diamond', '?'] ['what', 'would', 'happen', 'if', 'the', 'indian', 'government', 'stole', 'the', 'ko', '##hin', '##oor', '(', 'ko', '##h', '-', 'i', '-', 'no', '##or', ')', 'diamond', 'back', '?']
Token IDs:  [2054, 2003, 1996, 2466, 1997, 12849, 10606, 16506, 1006, 12849, 2232, 1011, 1045, 1011, 2053, 2953, 1007, 6323, 1029] [2054, 2052, 4148, 2065, 1996, 2796, 2231, 10312, 1996, 12849, 10606, 16506, 1006, 12849, 2232, 1011, 1045, 1011, 2053, 2953, 1007, 6323, 2067, 1029]
Combined:  [101, 2054, 2003, 1996, 2466, 1997, 12849, 10606, 16506, 1006, 12849, 2232, 1011, 1045, 1011, 2053, 2953, 1007, 6323, 1029, 102, 2054, 2052, 4148, 2065, 1996, 2796, 2231, 10312, 1996, 12849, 10606, 16506, 1006, 12849, 2232, 1011, 1045, 1011, 2053, 2953, 1007, 6323, 2067, 1029, 102]
Segmenta

In [15]:
print(len(q1))
print(len(q2))

9999
9999


In [16]:
print('Max sentence 1 length: ', max([len(sen) for sen in q1]))
print('Max sentence 2 length: ', max([len(sen) for sen in q2]))

Max sentence 1 length:  370
Max sentence 2 length:  1169


In [0]:
tk_1 = []
tk_2 = []
for i in range(len(q1)):
    if type(q1[i])==str and type(q2[i])==str:
        t1 = tokenizer.tokenize(q1[i])
        t2 = tokenizer.tokenize(q2[i])
        id1 = tokenizer.convert_tokens_to_ids(t1)
        id2 = tokenizer.convert_tokens_to_ids(t2)
        tk_1.append(id1)
        tk_2.append(id2)
tk_p1 = tf.keras.preprocessing.sequence.pad_sequences(tk_1, maxlen=500, dtype='long', truncating='post', padding='post', value=0)
tk_p2 = tf.keras.preprocessing.sequence.pad_sequences(tk_2, maxlen=500, dtype='long', truncating='post', padding='post', value=0)

In [0]:
input_ids = []
lab = []
segmentation_masks = []
for i in range(len(tk_p1)):
    encoded_pair = tokenizer.build_inputs_with_special_tokens(list(tk_p1[i]), list(tk_p2[i]))
    seg_mask = tokenizer.create_token_type_ids_from_sequences(tk_p1[i], tk_p2[i])
    #print(len(encoded_pair))
    input_ids.append(encoded_pair)
    segmentation_masks.append(seg_mask)
    lab.append(labels[i])

In [68]:
len(input_ids)

9999

In [0]:
attention_masks = []
for pair in input_ids:
    att_mask = [int(token_id > 0) for token_id in pair]    
    attention_masks.append(att_mask)

In [0]:
from sklearn.model_selection import train_test_split

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, lab, random_state=2018, test_size=0.3)
train_attn_masks, validation_attn_masks, _, _ = train_test_split(attention_masks, lab, random_state=2018, test_size=0.3)
train_seg_masks, validation_seg_masks, _, _ = train_test_split(segmentation_masks, lab, random_state=2018, test_size=0.3)

In [0]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_attn_masks = torch.tensor(train_attn_masks)
validation_attn_masks = torch.tensor(validation_attn_masks)

train_seg_masks = torch.tensor(train_seg_masks)
validation_seg_masks = torch.tensor(validation_seg_masks)

In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 16

train_data = TensorDataset(train_inputs, train_attn_masks, train_seg_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_attn_masks, validation_seg_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [75]:
from transformers import BertForSequenceClassification, BertConfig 

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                                                          num_labels = 2, 
                                                                                           output_attentions = False, 
                                                                                           output_hidden_states = False, 
                                                                                        )
model.cuda() 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 2e-5,  eps = 1e-8 )

In [0]:
import numpy as np
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed): #Takes a time in seconds and returns a string hh:mm:ss
    elapsed_rounded = int(round((elapsed)))    
    return str(datetime.timedelta(seconds=elapsed_rounded)) # Format as hh:mm:ss

In [81]:
import random

epochs = 10
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []
for epoch_i in range(epochs):
    t0 = time.time()
    total_loss = 0

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    model.train() # set model in train mode
    for step, batch in enumerate(train_dataloader):

        if step % 100 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_attn_mask = batch[1].to(device)
        b_input_seg_mask = batch[2].to(device)
        b_labels = batch[3].to(device)

        model.zero_grad()
        outputs = model(b_input_ids, token_type_ids=b_input_seg_mask, attention_mask=b_input_attn_mask, labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    avg_train_loss = total_loss / len(train_dataloader)    
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_attn_mask = batch[1].to(device)
        b_input_seg_mask = batch[2].to(device)
        b_labels = batch[3].to(device)
        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=b_input_seg_mask, attention_mask=b_input_attn_mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 10 ========
Training...


RuntimeError: ignored